In [1]:
# example.py
from collections import OrderedDict
import asyncio
from copra.websocket import Channel, Client

class Level2(Client):
    K=5
    Bids = {}
    Asks = {}
    reference_price = None
    
    def update_order_book(self):
        sorted_bids = list(reversed(sorted(self.Bids.items())))
        sorted_asks = list(sorted(self.Asks.items()))        
        best_bid = sorted_bids[0]
        best_ask = sorted_asks[0]
        if ((best_bid[0] + best_ask[0]) % 2) != 0:
            self.reference_price = round((best_bid[0]+best_ask[0])/2, 1)
        else:
            self.reference_price = round((best_bid[0]+best_ask[0])/2 + 0.5,1)  
    
    def print_order_book(self):
        print("Reference Price: {}".format(self.reference_price))
        print("-------------------")
        
        first_bid = int(round(self.reference_price - 0.5))
        first_ask = int(round(self.reference_price + 0.5))
        
        print("First {} Bids: ".format(self.K))
        for price in range(first_bid - self.K + 1, first_bid + 1):
            print("{}, {}".format(price, self.Bids.get(price, 0)))
        print("-------------------")
        
        print("First {} Asks: ".format(self.K))
        for price in range(first_ask, first_bid + self.K):
            print("{}, {}".format(price, self.Asks.get(price, 0)))
        print("\n")
                
    def on_message(self, message):
        if message['type'] == 'snapshot':
            for price, amount in message['bids']:
                self.Bids[int(round((float(price)*100)))] = float(amount)
            for price, amount in message['asks']:
                self.Asks[int(round((float(price)*100)))] = float(amount)
            self.update_order_book()
            self.print_order_book()
            
        if message['type'] == 'l2update' and 'time' in message:
            significant_order = False
            for (side, price, amount) in message['changes']:
                print("Update: {}".format((side, price, amount)))
                print("Time: {}".format(message['time']))
                print("-------------------")
                price = int(round((float(price)*100)))
                if abs(price - self.reference_price) < self.K:
                    significant_order = True
                    
                if side == "buy":
                    if amount == "0":
                        del self.Bids[price]
                    else:
                        self.Bids[price] = float(amount)
                elif side == "sell":
                    if amount == "0":
                        del self.Asks[price]
                    else:
                        self.Asks[price] = float(amount)           
                        
            self.update_order_book()
            if significant_order:
                self.print_order_book()
                
            
            
loop = asyncio.get_event_loop()
channel = Channel('level2', 'ETC-USD')
level2data = Level2(loop, channel)

try:
    loop.run_forever()
except KeyboardInterrupt:
    loop.run_until_complete(ws.close())
    loop.close()

Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1031.64339199
891, 2840.87001
892, 761.82894815
893, 99.25595848
-------------------
First 5 Asks: 
894, 841.1725308
895, 2865.36198217
896, 1954.96401289
897, 1357.547


Update: ('sell', '21.98000000', '1.3')
Time: 2018-10-31T03:56:22.434Z
-------------------
Update: ('sell', '13.30000000', '5.46228694')
Time: 2018-10-31T03:56:22.446Z
-------------------
Update: ('sell', '17.64000000', '4.34138694')
Time: 2018-10-31T03:56:22.448Z
-------------------
Update: ('buy', '4.56000000', '4.34138694')
Time: 2018-10-31T03:56:22.452Z
-------------------
Update: ('buy', '8.90000000', '1035.98477893')
Time: 2018-10-31T03:56:22.456Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1035.98477893
891, 2840.87001
892, 761.82894815
893, 99.25595848
-------------------
First 5 Asks: 
894, 841.1725308
895, 2865.36198217
896, 1954.96401289
897, 1357.547


Update: ('buy', '

Update: ('sell', '8.94000000', '841.1725308')
Time: 2018-10-31T03:56:24.558Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1031.64339199
891, 2840.87001
892, 761.82894815
893, 99.25595848
-------------------
First 5 Asks: 
894, 841.1725308
895, 2865.36198217
896, 1954.96401289
897, 1357.547


Update: ('buy', '4.56000000', '0')
Time: 2018-10-31T03:56:24.599Z
-------------------
Update: ('buy', '0.22000000', '5000')
Time: 2018-10-31T03:56:24.602Z
-------------------
Update: ('sell', '30.66000000', '4.34138694')
Time: 2018-10-31T03:56:24.608Z
-------------------
Update: ('sell', '26.32000000', '4.34138694')
Time: 2018-10-31T03:56:24.613Z
-------------------
Update: ('sell', '8.94000000', '845.51391774')
Time: 2018-10-31T03:56:25.056Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1031.64339199
891, 2840.87001
892, 761.82894815
893, 99.25595848
-------------------
First 5 Asks: 
8

Update: ('sell', '17.64000000', '0')
Time: 2018-10-31T03:56:26.397Z
-------------------
Update: ('buy', '0.22000000', '5000')
Time: 2018-10-31T03:56:26.437Z
-------------------
Update: ('buy', '4.56000000', '0')
Time: 2018-10-31T03:56:26.437Z
-------------------
Update: ('sell', '8.94000000', '841.1725308')
Time: 2018-10-31T03:56:26.438Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1031.64339199
891, 2840.87001
892, 761.82894815
893, 99.25595848
-------------------
First 5 Asks: 
894, 841.1725308
895, 2865.36198217
896, 1954.96401289
897, 1357.547


Update: ('sell', '13.30000000', '1.1209')
Time: 2018-10-31T03:56:26.441Z
-------------------
Update: ('sell', '21.98000000', '1.3')
Time: 2018-10-31T03:56:26.480Z
-------------------
Update: ('buy', '8.90000000', '1035.98477893')
Time: 2018-10-31T03:56:26.484Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1035.98477893
891, 2840.

Update: ('sell', '13.30000000', '5.46228694')
Time: 2018-10-31T03:56:28.702Z
-------------------
Update: ('buy', '8.90000000', '1031.64339199')
Time: 2018-10-31T03:56:28.734Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1031.64339199
891, 2840.87001
892, 761.82894815
893, 99.25595848
-------------------
First 5 Asks: 
894, 845.51391774
895, 2865.36198217
896, 1954.96401289
897, 1357.547


Update: ('buy', '0.22000000', '5004.34138694')
Time: 2018-10-31T03:56:28.740Z
-------------------
Update: ('sell', '17.64000000', '4.34138694')
Time: 2018-10-31T03:56:28.745Z
-------------------
Update: ('buy', '4.56000000', '4.34138694')
Time: 2018-10-31T03:56:28.749Z
-------------------
Update: ('sell', '8.94000000', '841.1725308')
Time: 2018-10-31T03:56:28.774Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1031.64339199
891, 2840.87001
892, 761.82894815
893, 99.25595848
-----------------

Update: ('sell', '21.98000000', '5.64138694')
Time: 2018-10-31T03:56:30.999Z
-------------------
Update: ('sell', '26.32000000', '4.34138694')
Time: 2018-10-31T03:56:31.036Z
-------------------
Update: ('buy', '0.22000000', '5000')
Time: 2018-10-31T03:56:31.112Z
-------------------
Update: ('sell', '8.94000000', '845.51391774')
Time: 2018-10-31T03:56:32.217Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1031.64339199
891, 2840.87001
892, 761.82894815
893, 99.25595848
-------------------
First 5 Asks: 
894, 845.51391774
895, 2865.36198217
896, 1954.96401289
897, 1357.547


Update: ('buy', '8.90000000', '1035.98477893')
Time: 2018-10-31T03:56:32.259Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1260.62452843
890, 1035.98477893
891, 2840.87001
892, 761.82894815
893, 99.25595848
-------------------
First 5 Asks: 
894, 845.51391774
895, 2865.36198217
896, 1954.96401289
897, 1357.547


Update: ('sel

...

Update: ('sell', '8.94000000', '1730.01336733')
Time: 2018-10-31T03:58:45.477Z
-------------------
Reference Price: 892.5
-------------------
First 5 Bids: 
888, 282.84095225
889, 1516.71252843
890, 1190.47039199
891, 2845.21139694
892, 883.63434815
-------------------
First 5 Asks: 
893, 11.2
894, 1730.01336733
895, 2496.30198217
896, 1454.96401289


Update: ('buy', '0.22000000', '5000')
Time: 2018-10-31T03:58:45.483Z
-------------------
Update: ('sell', '17.64000000', '0')
Time: 2018-10-31T03:58:45.484Z
-------------------
Update: ('buy', '4.56000000', '0')
Time: 2018-10-31T03:58:45.485Z
-------------------
Update: ('buy', '8.91000000', '2840.87001')
Time: 2018-10-31T03:58:45.486Z
-------------------
Reference Price: 892.5
-------------------
First 5 Bids: 
888, 282.84095225
889, 1516.71252843
890, 1190.47039199
891, 2840.87001
892, 883.63434815
-------------------
First 5 Asks: 
893, 11.2
894, 1730.01336733
895, 2496.30198217
896, 1454.96401289


Update: ('sell', '21.98000000', '5.6

Update: ('sell', '13.30000000', '5.46228694')
Time: 2018-10-31T03:58:48.884Z
-------------------
Update: ('sell', '13.30000000', '1.1209')
Time: 2018-10-31T03:58:49.120Z
-------------------
Update: ('sell', '17.64000000', '0')
Time: 2018-10-31T03:58:49.161Z
-------------------
Update: ('sell', '21.98000000', '5.64138694')
Time: 2018-10-31T03:58:49.171Z
-------------------
Update: ('sell', '26.32000000', '0')
Time: 2018-10-31T03:58:49.445Z
-------------------
Update: ('sell', '30.66000000', '0')
Time: 2018-10-31T03:58:49.449Z
-------------------
Update: ('sell', '13.30000000', '5.46228694')
Time: 2018-10-31T03:58:49.461Z
-------------------
Update: ('sell', '8.94000000', '1731.45975427')
Time: 2018-10-31T03:58:49.464Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1516.71252843
890, 1190.47039199
891, 2840.87001
892, 1001.73434815
893, 0
-------------------
First 5 Asks: 
894, 1731.45975427
895, 2508.70198217
896, 1454.96401289
897, 3297.211


Update:

Update: ('sell', '30.66000000', '4.34138694')
Time: 2018-10-31T03:58:53.221Z
-------------------
Update: ('sell', '35.00000000', '338.95637334')
Time: 2018-10-31T03:58:53.260Z
-------------------
Update: ('sell', '26.32000000', '0')
Time: 2018-10-31T03:58:53.540Z
-------------------
Update: ('buy', '0.22000000', '5000')
Time: 2018-10-31T03:58:53.580Z
-------------------
Update: ('sell', '39.34000000', '4.34138694')
Time: 2018-10-31T03:58:53.588Z
-------------------
Update: ('sell', '43.69000000', '4.34138694')
Time: 2018-10-31T03:58:53.592Z
-------------------
Update: ('sell', '30.66000000', '0')
Time: 2018-10-31T03:58:53.662Z
-------------------
Update: ('sell', '35.00000000', '334.6149864')
Time: 2018-10-31T03:58:53.665Z
-------------------
Update: ('buy', '8.91000000', '2845.21139694')
Time: 2018-10-31T03:58:53.672Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1516.71252843
890, 1190.47039199
891, 2845.21139694
892, 1001.73434815
893, 0
--------

Update: ('sell', '8.94000000', '1727.11836733')
Time: 2018-10-31T03:58:56.077Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1516.71252843
890, 1190.47039199
891, 2840.87001
892, 1001.73434815
893, 0
-------------------
First 5 Asks: 
894, 1727.11836733
895, 2501.60198217
896, 1454.96401289
897, 3297.211


Update: ('sell', '8.95000000', '2504.80198217')
Time: 2018-10-31T03:58:56.086Z
-------------------
Reference Price: 893.5
-------------------
First 5 Bids: 
889, 1516.71252843
890, 1190.47039199
891, 2840.87001
892, 1001.73434815
893, 0
-------------------
First 5 Asks: 
894, 1727.11836733
895, 2504.80198217
896, 1454.96401289
897, 3297.211


Update: ('buy', '4.56000000', '4.34138694')
Time: 2018-10-31T03:58:56.125Z
-------------------
Update: ('sell', '13.30000000', '1.1209')
Time: 2018-10-31T03:58:56.322Z
-------------------
Update: ('buy', '0.22000000', '5000')
Time: 2018-10-31T03:58:56.361Z
-------------------
Update: ('sell', '21.98000000', '

NameError: name 'ws' is not defined